In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

tesr_dataset = datasets.MNIST(root='data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=False)

def hot_one_encoding(label):
    encoding = torch.zeros(10)
    encoding[label] = 1.0
    return encoding

def prepare_data(images, labels):
    inputs = []
    targets = []

    for image, label in zip(images, labels):
        hot_one_label = hot_one_encoding(label)

        wrong_label = (label + 1) % 10
        wrong_hot_one_label = hot_one_encoding(wrong_label)

        inputs.append((image, hot_one_label))
        inputs.append((image, wrong_hot_one_label))

        targets.append(hot_one_label)
        targets.append(wrong_hot_one_label)

    return inputs, targets

sample_inputs, sample_targets = prepare_data(*next(iter(train_loader)))

print("Sample Input:", sample_inputs[0])
print("Sample Target:", sample_targets[0])


Sample Input: (tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.00

In [44]:
class PositiveLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super(PositiveLayer, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear(x)
        return x
class NegativeLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super(NegativeLayer, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear(x)
        return x

class LearningNetwork(nn.Module):
    def __init__(self, positive_layer, negative_layer):
        super(LearningNetwork, self).__init__()
        self.positive_layer = positive_layer
        self.negative_layer = negative_layer

    def forward(self, positive_data, negative_data):
        positive_output = self.positive_layer(positive_data)
        negative_output = self.negative_layer(negative_data)
        return positive_output, negative_output

def custom_loss(threshold, positive_data, negative_data):
    return torch.mean(torch.log(1 + torch.exp(torch.cat([threshold - positive_data, negative_data - threshold]))))

input_size = 28 * 28
output_size = 10
threshold = 0.5
positive_layer = PositiveLayer(input_size, output_size)
negative_layer = NegativeLayer(input_size, output_size)
model = LearningNetwork(positive_layer, negative_layer)

optimizer = optim.SGD(model.parameters(), lr=0.001)

def train(model, optimizer, num_epochs):
    for epoch in range(num_epochs):
        for batch_images, batch_labels in train_loader:
            inputs, targets = prepare_data(batch_images, batch_labels)

            positive_data, _ = inputs[0]
            negative_data, _ = inputs[1]

            positive_output, negative_output = model(positive_data.flatten(), negative_data.flatten())
            loss = custom_loss(threshold, positive_output, negative_output)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

num_epochs = 10
train(model, optimizer, num_epochs)

Epoch 1/10, Loss: 0.050567857921123505
Epoch 2/10, Loss: 0.019389096647500992
Epoch 3/10, Loss: 0.008798522874712944
Epoch 4/10, Loss: 0.01200067438185215
Epoch 5/10, Loss: 0.007391289807856083
Epoch 6/10, Loss: 0.0076102628372609615
Epoch 7/10, Loss: 0.007415018975734711
Epoch 8/10, Loss: 0.004356746096163988
Epoch 9/10, Loss: 0.008527066558599472
Epoch 10/10, Loss: 0.0043122125789523125


The loss function tries to penalize deviations from the desired conditions. It encourages the model to assign high values to positive data (where
threshold - positive data
threshold - positive data is negative) and low values to negative data (where
negative data - threshold
negative data - threshold is negative). The logarithmic term ensures that the penalty is better for larger deviations.

In [45]:
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            inputs, targets = prepare_data(images, labels)

            positive_data, _ = inputs[0]
            negative_data, _ = inputs[1]

            positive_output, negative_output = model(positive_data.flatten(), negative_data.flatten())

            # Assuming positive_output is a 1D tensor
            predicted_labels = torch.argmax(positive_output)

            correct += (predicted_labels == labels[0]).item()
    size = len(data_loader.dataset)
    accuracy = correct / size
    return accuracy

# Evaluate the model on the training set
train_accuracy = evaluate(model, train_loader)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

Training Accuracy: 0.19%


Loading the MNIST dataset.

Generating data by converting labels into one-hot vectors and placing them in the images.

Creating pairs of correct and incorrect labels for each image.

Positive and Negative layers are created as linear layers.

The Learning Network is composed of these layers.

Custom loss is defined to have the goodness conditions, with a threshold for positive and negative data.

The loss function encourages correct prediction for positive data and incorrect prediction for negative data.

Stochastic Gradient Descent (SGD) is employed as the optimizer.

Training involves iterating through epochs and batches, forward-passing positive and negative data through the model, calculating the loss, and updating the model parameters.

After training, the model's performance saved by evaluating its accuracy on the training dataset.